In [1]:
import data_analysis_tools as da

data = da.file_ops.read_csv('./files/Iris.csv', sep=',', as_array=True)

# some formatting
data.pop(0)
for i in range(len(data)):
    data[i].pop(0)
    data[i] = ([data[i][0], data[i][2]], data[i][4])
data[:10]

[(['5.1', '1.4'], 'Iris-setosa'),
 (['4.9', '1.4'], 'Iris-setosa'),
 (['4.7', '1.3'], 'Iris-setosa'),
 (['4.6', '1.5'], 'Iris-setosa'),
 (['5.0', '1.4'], 'Iris-setosa'),
 (['5.4', '1.7'], 'Iris-setosa'),
 (['4.6', '1.4'], 'Iris-setosa'),
 (['5.0', '1.5'], 'Iris-setosa'),
 (['4.4', '1.4'], 'Iris-setosa'),
 (['4.9', '1.5'], 'Iris-setosa')]

In [ ]:
da.r.seed(0)
cls = da.ml.KMeans(k=3)
cls.train()